## Utilizando Web-Scraping para fazer pesquisa de preço

#### Passo 1: instalar bibliotecas

In [ ]:
# Remova o comentário(#) da linha abaixo caso esteja executando este script pela primeira vez
#!pip install requests
#!pip install beautifulsoup4

#### Passo 2: Importar bibliotecas

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#### Passo 3: Formatar link de busca

In [ ]:
# Copie o modelo do link de busca
url_de_busca = 'https://lista.mercadolivre.com.br/'

# Informe a palvra chave para busca
palavra_chave = 'smart tv 70'

# Formata a palavra chave conforme modelo do site de busca
palavra_chave = palavra_chave.replace(' ','-')

# Formata link
link = url_de_busca+palavra_chave
print(link)

#### Passo 4: Obter informações do link

In [ ]:
# Requisita informações do site
response = requests.get(link)
# Cria um objeto BeautifulSoup com as informações html
soup = BeautifulSoup(response.content, 'html.parser')

#### Passo 5: Encontrar elementos

In [ ]:
# Encontra as informações contidas na tag de resultado de busca
resultados = soup.find_all('div', {'class': 'ui-search-result__content'})
# Conta quantos resultados apareceram na primeira página
len(resultados)

#### Passo 6: Visualizar links dos produtos

In [ ]:
# Criar uma lista vazia para guardar dados
lista_de_links = []

# Criar laço de repetição para filtrar todos os dados
for i in resultados:
    lista_de_links.append(i.find('a')['href'])

lista_de_links

#### Passo 7: Extrarir informações de cada produto listado

In [ ]:
# Requisita informações do link do produto e cria objeto BeautifulSoup
produto = BeautifulSoup(requests.get(lista_de_links[0]).content, 'html.parser')

In [ ]:
# Seleciona o título
produto.find('h1', attrs={'class': 'ui-pdp-title'}).text

In [ ]:
# Seleciona o preço
produto.find('span', attrs={'class': 'andes-money-amount__fraction'}).text

In [ ]:
# Seleciona a quantidade de vendas
produto.find('span', attrs={'class': 'ui-pdp-subtitle'}).text

In [ ]:
# Seleciona a descrição
produto.find('p', attrs={'class': 'ui-pdp-description__content'}).text

#### Criar tabela para armazenar dados

In [ ]:
# Criar o nome de cada coluna
colunas = ['Título','Preço','Quantidade de Vendas','Descrição do produto']

# Criar a tabela com as colunas informadas
tabela = pd.DataFrame(columns=colunas)

# Visualizar tabela
tabela

In [ ]:
# Criar laço de repetição para armazenar as informações de todos os produtos da lista de links

id = 0 # Contador
for j in lista_de_links:
    try:
        # Requisita informações do link do produto e cria objeto BeautifulSoup
        produto = BeautifulSoup(requests.get(j).content, 'html.parser')
    
        # Seleciona o título
        titulo = produto.find('h1', attrs={'class': 'ui-pdp-title'}).text
        # Seleciona o preço
        preco = produto.find('span', attrs={'class': 'andes-money-amount__fraction'}).text
        # Seleciona a quantidade de vendas
        vendidos = produto.find('span', attrs={'class': 'ui-pdp-subtitle'}).text
        # Seleciona a descrição
        descricao = produto.find('p', attrs={'class': 'ui-pdp-description__content'}).text

        # Insere uma nova linha na tabela com as informações do produto
        tabela.loc[len(tabela)] = [titulo,preco,vendidos,descricao]

    except:
        print(f'Houve um problema no link de índice: {id}')
    
    # Incrementa uma posição no contador
    id = id+1

In [ ]:
# Visualizar tabela preenchida
tabela

#### Passo 8: Salvar tabela em uma planilha de Excel

In [ ]:
# Salvar DataFrame
tabela.to_excel('Lista de produtos.xlsx')

#### Extra: formatar dados da tabela

In [ ]:
import re 

id = 0
for i in tabela['Quantidade de Vendas']:
    tabela['Quantidade de Vendas'][id] = re.sub('[^0-9]', '', i)
    id = id+1
tabela

### Exercício: Encontrar uma solução para listar todos os resultados quando houver 2 ou mais páginas de resultado